# Now, lets focus on getting our data ready, so we could make embeddings. 
- Since we are predicting the 'next word' in a sentence. We do not have any default labels.
- We will be using our input text to create a label. While creating a dataset, during each iteration, the output for the previous iteration will be a part of the input in our current iteration.
- We will use a sliding-window approach to achive this. 

In [83]:
import tiktoken 


In [84]:
tokenizer = tiktoken.get_encoding("gpt2")

In [85]:
with open('dprpht.txt', 'r', encoding='utf-8') as f:
    raw_data = f.read()
token_ids = tokenizer.encode(raw_data)
print(len(token_ids))

23470


In [86]:
sample_enc = token_ids[50:]
# we will just skip first 50 tokenIds (this is just to remove the unnecessary data at the top of the file)

### Lets, see an example to see how we create inputs and output labels. 
- context_len is just the number of tokens we will see at a time.
- lets take context_len = 4, for example. 

In [87]:
context_len = 4 
x = sample_enc[:context_len]     #input text
y = sample_enc[1:context_len+1]  #corresponding output labels. In the 
print(f"{x}")
print(f"   {y}")

[743, 4866, 340, 11]
   [4866, 340, 11, 1577]


In [88]:
for i in range(1, context_len+1):
    context = sample_enc[:i]
    desired = sample_enc[i]
    print(context, "---->", desired)

[743] ----> 4866
[743, 4866] ----> 340
[743, 4866, 340] ----> 11
[743, 4866, 340, 11] ----> 1577


In [89]:
for i in range(1, context_len+1):
    context = sample_enc[:i]
    desired = sample_enc[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 may ---->  copy
 may copy ---->  it
 may copy it ----> ,
 may copy it, ---->  give


### Now that we know how our inputs and labels look, lets create a Dataset and a DataLoader using PyTorch
- A custom Dataset class must implement three functions: \__init\_\_, \__len\_\_ , and \__getitem\_\_.

In [90]:
import torch
from torch.utils.data import Dataset, DataLoader

In [91]:
#Implementing a custom Dataset Class
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        # text : The document we want to use to create our dataset
        # tokenizer : BPE tokenizer that we imported from tiktoken
        # max_length : Context length (no. of token our model can see)
        # stride : The number of units our next input should be ahead of our current input 
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        vocab_size = tokenizer.n_vocab
        print(f"vocab_size: {vocab_size}")
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            output_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(output_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.output_ids[idx]
        
        

### At the heart of PyTorch data loading utility is the torch.utils.data.DataLoader class. It represents a Python iterable over a dataset, with support for

- map-style and iterable-style datasets,
- customizing data loading order,
- automatic batching,
- single- and multi-process data loading,
- automatic memory pinning.
- These options are configured by the constructor arguments of a DataLoader, which has signature:
```
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, *, prefetch_factor=2,
           persistent_workers=False)
```

In [92]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    #initialize tokenizer 
    tokenizer = tiktoken.get_encoding("gpt2")
    #create Dataset
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers) 
    return dataloader

In [93]:
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK THE PROPHET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK THE PROPHET ***"

start_idx = raw_data.find(start_marker)
end_idx = raw_data.find(end_marker)

if start_idx == -1 or end_idx == -1:
    raise ValueError("Start or end index not found")
   
# Slice the content between markers
trimmed_raw_data = raw_data[start_idx + len(start_marker):end_idx]


In [94]:
dataloader = create_dataloader_v1(trimmed_raw_data, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)

vocab_size: 50257


In [95]:
print(first_batch)

[tensor([[  628,   628,   198, 10970]]), tensor([[  628,   198, 10970,  4810]])]


In [96]:
second_batch = next(data_iter)

In [97]:
second_batch

[tensor([[  628,   198, 10970,  4810]]),
 tensor([[  198, 10970,  4810,  3185]])]

In [98]:
dataloader = create_dataloader_v1(trimmed_raw_data, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

vocab_size: 50257
Inputs:
 tensor([[  628,   628,   198, 10970],
        [ 4810,  3185,    39,  2767],
        [  198,   198,  3886,   509],
        [15668,   346, 12488,  2596],
        [  198,   198,  3791,  1971],
        [   25, 22044,   317,    13],
        [ 6102,   404,    69,   198],
        [  198,  1129,  1954,   198]])

Targets:
 tensor([[  628,   198, 10970,  4810],
        [ 3185,    39,  2767,   198],
        [  198,  3886,   509, 15668],
        [  346, 12488,  2596,   198],
        [  198,  3791,  1971,    25],
        [22044,   317,    13,  6102],
        [  404,    69,   198,   198],
        [ 1129,  1954,   198,   198]])


Let's illustrate how the token ID to embedding vector conversion works with a hands-on example. Suppose we have the following four input tokens with IDs 2, 3, 5, and 1:


In [99]:
input_ids = torch.tensor([2,3,5,1])


For the sake of simplicity and illustration purposes, suppose we have a small vocabulary of only 6 words (instead of the 50,257 words in the BPE tokenizer vocabulary), and we want to create embeddings of size 3 (in GPT-3, the embedding size is 12,288 dimensions):

Using the vocab_size and output_dim, we can instantiate an embedding layer in PyTorch, setting the random seed to 123 for reproducibility purposes:

In [100]:
vocab_size = 6 
ndim  = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, ndim)
embedding_layer

Embedding(6, 3)

In [101]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


After we instantiated the embedding layer, let's now apply it to a token ID to obtain the
embedding vector:


In [102]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [103]:
print(embedding_layer(torch.tensor(input_ids)))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


/var/folders/64/_jgz4l191qvct7dgtf2wljyr0000gn/T/ipykernel_86972/3846183200.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(embedding_layer(torch.tensor(input_ids)))


In [107]:
max_length = 4
dataloader = create_dataloader_v1(trimmed_raw_data, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, labels = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

vocab_size: 50257
Token IDs:
 tensor([[  628,   628,   198, 10970],
        [ 4810,  3185,    39,  2767],
        [  198,   198,  3886,   509],
        [15668,   346, 12488,  2596],
        [  198,   198,  3791,  1971],
        [   25, 22044,   317,    13],
        [ 6102,   404,    69,   198],
        [  198,  1129,  1954,   198]])

Inputs shape:
 torch.Size([8, 4])


In [108]:
#lets create embeddings layer weight matrix for our original dataset and add positional embedding those embeddings. 
vocab_size = 50257
ndim = 256 
token_embedding_layer = torch.nn.Embedding(vocab_size, ndim)
print(token_embedding_layer.weight[:4])

tensor([[ 0.7481, -1.0844,  0.8362,  ..., -0.1661,  0.0175, -0.7167],
        [-0.1840,  0.7819, -0.0508,  ..., -0.0145, -0.5577, -0.6014],
        [ 0.9606, -0.3260, -1.4057,  ...,  0.7417, -0.1273,  0.2401],
        [ 1.2456,  0.7682, -0.3834,  ...,  0.5522,  0.2447,  0.7976]],
       grad_fn=<SliceBackward0>)


In [109]:
#lets create embeddings for each input token 
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [112]:
# now lets create positional embeddings to add to each token (4 x 256). 
context_length = max_length 
pos_embedding_layer = torch.nn.Embedding(context_length, ndim) 
print(pos_embedding_layer.weight.shape)

torch.Size([4, 256])


In [113]:
# we have 4 positions (4 is the length of our context) so lets create embedding for each position
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [114]:
#Now lets add posistional embeddings to token embeddings
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
